In [1]:
import scrapy
from scrapy.crawler import CrawlerRunner, CrawlerProcess

from crochet import setup
setup()

import re

import requests
from scrapy.http import TextResponse

In [14]:
import pandas as pd
import numpy as np

2022-09-27 15:45:02 [numexpr.utils] INFO: NumExpr defaulting to 8 threads.


In [5]:
url = 'https://www.sciencedirect.com/science/article/abs/pii/S0921344920301816'
r = requests.get(url)
response = TextResponse(r.url, body=r.text,encoding='utf-8')

2022-09-27 15:07:58 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.sciencedirect.com:443
2022-09-27 15:07:59 [urllib3.connectionpool] DEBUG: https://www.sciencedirect.com:443 "GET /science/article/abs/pii/S0921344920301816 HTTP/1.1" 403 None


In [6]:
response

<200 https://www.sciencedirect.com/science/article/abs/pii/S0921344920301816>

In [7]:
# get heading-css
response.css("a").extract_first()

'<a id="gh-branding" class="u-flex-center-ver" href="https://www.sciencedirect.com/" aria-label="Science Direct home page" data-aa-region="header" data-aa-name="ScienceDirect">\n\n<svg xmlns="http://www.w3.org/2000/svg" role="img" version="1.1" height="30" width="138" viewbox="0 0 138 30" class="gh-wordmark u-margin-s-left" fill="#f36d21" aria-labelledby="gh-wm-science-direct" focusable="false" aria-hidden="true">\n<title id="gh-wm-science-direct">Science Direct</title>\n<g>\n<path class="a" d="M4.23,21a9.79,9.79,0,0,1-4.06-.83l.29-2.08a7.17,7.17,0,0,0,3.72,1.09c2.13,0,3-1.22,3-2.39C7.22,13.85.3,13.43.3,9c0-2.37,1.56-4.29,5.2-4.29a9.12,9.12,0,0,1,3.77.75l-.1,2.08a7.58,7.58,0,0,0-3.67-1c-2.24,0-2.91,1.22-2.91,2.39,0,3,6.92,3.61,6.92,7.8C9.5,19.1,7.58,21,4.23,21Z">\n</path>\n<path class="a" d="M20.66,20A6.83,6.83,0,0,1,16.76,21c-3,0-5.23-2.18-5.23-6.29,0-4.29,2.91-6.11,5.28-6.11,2.16,0,3.67.55,3.85,2.11,0,.23,0,.57,0,.86H18.81c0-1-.55-1.25-1.9-1.25a2.87,2.87,0,0,0-1.35.21c-.21.13-1.85.94

In [8]:
# get heading xpath
response.xpath("//a").extract_first()

'<a id="gh-branding" class="u-flex-center-ver" href="https://www.sciencedirect.com/" aria-label="Science Direct home page" data-aa-region="header" data-aa-name="ScienceDirect">\n\n<svg xmlns="http://www.w3.org/2000/svg" role="img" version="1.1" height="30" width="138" viewbox="0 0 138 30" class="gh-wordmark u-margin-s-left" fill="#f36d21" aria-labelledby="gh-wm-science-direct" focusable="false" aria-hidden="true">\n<title id="gh-wm-science-direct">Science Direct</title>\n<g>\n<path class="a" d="M4.23,21a9.79,9.79,0,0,1-4.06-.83l.29-2.08a7.17,7.17,0,0,0,3.72,1.09c2.13,0,3-1.22,3-2.39C7.22,13.85.3,13.43.3,9c0-2.37,1.56-4.29,5.2-4.29a9.12,9.12,0,0,1,3.77.75l-.1,2.08a7.58,7.58,0,0,0-3.67-1c-2.24,0-2.91,1.22-2.91,2.39,0,3,6.92,3.61,6.92,7.8C9.5,19.1,7.58,21,4.23,21Z">\n</path>\n<path class="a" d="M20.66,20A6.83,6.83,0,0,1,16.76,21c-3,0-5.23-2.18-5.23-6.29,0-4.29,2.91-6.11,5.28-6.11,2.16,0,3.67.55,3.85,2.11,0,.23,0,.57,0,.86H18.81c0-1-.55-1.25-1.9-1.25a2.87,2.87,0,0,0-1.35.21c-.21.13-1.85.94

In [9]:
#get authors-css
response.css("small::text").extract()

[]

In [ ]:
#authors-xpath
response.xpath("//small/text()").extract()

In [ ]:
#heading - css
response.css('a[style="text-decoration: none"]').extract()

In [ ]:
#heading - css text only
response.css('a[style="text-decoration: none"]::text').extract()

In [ ]:
#headding - css href only
response.css('a[style="text-decoration: none"]::attr(href)').extract()

In [ ]:
#tag text css
response.css("a[class='tag']").extract()

In [ ]:
#tag text css
response.css("a[class='tag']::text").extract()

In [ ]:
#tag url css
response.css("a[class='tag']::attr(href)").extract()

In [ ]:
#tag text xpath
response.xpath("//a[@class='tag']/@href").extract()

In [ ]:
response.css("title").extract_first()

In [ ]:
response.css("title").re("title")

In [ ]:
#regex to get text between tags
response.css("title").re('.+>(.+)<.+')

# Demo spyder build

In [2]:
from crochet import setup, wait_for
setup()

In [12]:
class QuotesToCsv(scrapy.Spider):
    """scrape first line of  quotes from `wikiquote` by 
    Maynard James Keenan and save to json file"""
    name = "MJKQuotesToCsv"
    start_urls = [
        'https://en.wikiquote.org/wiki/Maynard_James_Keenan',
    ]
    custom_settings = {
        'ITEM_PIPELINES': {
            '__main__.ExtractFirstLine': 1
        },
        'FEEDS': {
            'quotes.csv': {
                'format': 'csv',
                'overwrite': True
            }
        }
    }

    def parse(self, response):
        """parse data from urls"""
        for quote in response.css('div.mw-parser-output > ul > li'):
            yield {'quote': quote.extract()}


class ExtractFirstLine(object):
    def process_item(self, item, spider):
        """text processing"""
        lines = dict(item)["quote"].splitlines()
        first_line = self.__remove_html_tags__(lines[0])

        return {'quote': first_line}

    def __remove_html_tags__(self, text):
        """remove html tags from string"""
        html_tags = re.compile('<.*?>')
        return re.sub(html_tags, '', text)

@wait_for(10)
def run_spider():
    """run spider with MJKQuotesToCsv"""
    crawler = CrawlerRunner()
    d = crawler.crawl(QuotesToCsv)
    return d

In [13]:
run_spider()

2022-09-27 15:42:58 [scrapy.crawler] INFO: Overridden settings:
{}
2022-09-27 15:42:58 [scrapy.extensions.telnet] INFO: Telnet Password: 857de5b0de53480c
2022-09-27 15:42:58 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2022-09-27 15:42:58 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermid

In [15]:
pd.read_csv('quotes.csv')

,quote
0,Tool is not Slayer. I went to art school. I sp...
1,I think there’s a reason why wine figures into...
2,You can grow grapes in almost any part of the ...
3,"For the music, it’s not about the individual —..."
4,It’s in my blood. My great-grandfather made wi...
5,"It's the role of us to run our government, the..."
6,"Every now and then, you get people who tend to..."
7,The army influences everything I do. Certainly...
8,The process that we go through in recording wi...
9,One of my biggest heroes in music has been Dav...
